In [1]:
!pip install Faker
!pip install pandas 
!pip install tweepy
!pip install confluent_kafka

     |████████████████████████████████| 1.2 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 8.5 MB/s  eta 0:00:01
     |████████████████████████████████| 2.7 MB 1.1 MB/s eta 0:00:01     |███████████████████████▎        | 2.0 MB 1.1 MB/s eta 0:00:01


In [1]:
import time

from confluent_kafka import Producer
import json
import tweepy

producer = Producer(**{"bootstrap.servers": "kafka2:19092"})

In [2]:
consumer_key = "hK08N3zIwkJLrF6BVK52b3aS8"
consumer_secret = "8M22ppkZzb3sC9u4acUZK5IBr9nsLSByctTOoUI3Y5THEMRk7s"
access_token = "774855303321886720-JxdOM3ieqEVck6l0rYJmSIV3A412anW"
access_token_secret = "AoJgcvbtWmikzHaoOwH2rURrzoRwbBGgoV4oEASumNMzF"

In [4]:
# Words to track
WORDS = ['#coronavirus', '#COVID-19', '#COVID19', '#COVID'] #, '#SocialDistancing', '#pandemic']
raw_tweets_topic = "1_RAW_tweets"

In [5]:
class StreamListener(tweepy.StreamListener):
    # This is a class provided by tweepy to access the Twitter Streaming API.

    def on_connect(self):
        # Called initially to connect to the Streaming API
        print("You are now connected to the streaming API.")

    def on_error(self, status_code):
        # On error - if an error occurs, display the error / status code
        print("Error received in kafka producer " + repr(status_code))
        
        return True 
    
    def preprocess(self, tweetData):
        userID = '-'
        tweetText = '-'
        hashTags = []
        location_full_name = '-'
        created_at = '-'
        favoriteCount = 0
        reTweetCount = 0
        

        try:
            userID = tweetData["user"]["id"]
            
            if tweetData["truncated"] == True:
                tweetText = tweetData["extended_tweet"]["full_text"]
                hashTags = [ hTag["text"] for hTag in tweetData["extended_tweet"]["entities"]["hashtags"] ]
            else:
                tweetText = tweetData["text"]
                hashTags = [ hTag["text"] for hTag in tweetData["entities"]["hashtags"] ]

            if hashTags == []:
                return {}

            if "user" in tweetData and tweetData["user"] != None:
                location_full_name = tweetData["user"]["location"]
                
            created_at = tweetData["created_at"]
            favoriteCount = tweetData["favorite_count"]
            reTweetCount = tweetData["retweet_count"]
            

            rec = {'userID': userID, 
                   'tweetText': tweetText, 
                   'hashTags': hashTags, 
                   'location_full_name': location_full_name,
                   'favoriteCount': favoriteCount,
                   'reTweetCount': reTweetCount,
                   'created_at': created_at}

            return json.dumps(rec)

        except Exception as ex:
            print('Exception while parsing')
            print(str(ex))

    def on_data(self, data):
        """ This method is called whenever new data arrives from live stream.
        We asynchronously push this data to kafka queue"""
        try:
            parsed = json.loads(data)

            
            if "user" in parsed and "location" in parsed["user"]:
                if parsed["user"]["location"] != None:

                    processed = self.preprocess(parsed)

                    if isinstance(processed, str):
                        producer.produce(raw_tweets_topic, value=processed.encode('utf-8'))
                        producer.flush()

                        print(parsed["text"])

        except Exception as e:
            print("error! : " + str(e))
            return False #stop stream

        return True

    def on_timeout(self):
        return True

In [ ]:
# Create Auth object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Set up the listener. The 'wait_on_rate_limit=True' is needed to help with Twitter API rate limiting.
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=60, retry_delay=5, retry_count=10, retry_errors=set([401, 404, 500, 503])))
stream = tweepy.Stream(auth=auth, listener=listener)
print("Tracking: " + str(WORDS))
stream.filter(track=WORDS, languages = ['en'])

Tracking: ['#coronavirus', '#COVID-19', '#COVID19', '#COVID']
You are now connected to the streaming API.
RT @WIONews: Indian authorities are restricting religious festivals due to the threat of the outbreak of #COVID19's third wave. Several gov…
Woot! Fully vaccinated and ready to kick #COVID19 in the ghoulies. https://t.co/ITPEofGP0D
RT @carolJhedges: 😬Is it just my deeply suspicious mind, or is this 'let's fund the #NHS &amp; #socialcare' actually a ruse to grab a refund fo…
According to predictions made about two weeks ago by Hebrew University researchers, the #coronavirus infection rate… https://t.co/SeUR1AZDhU
RT @OrganicConsumer: Factory pig farms are spreading #COVID19: ➡️  "This has to do with two things: unhealthy human labor and poor conditio…
RT @DrEricDing: Wow—“Every kindegarten teacher” at Kinder Ranch Elementary in @cisdnews tested positive for #COVID19. However, the school s…
We've crossed the 14 million mark, #Mzansi. 👏🏾 If you can help someone get their jab, please d